In [1]:
import sys

sys.path.insert(0, '..')
import os
from biosimulator_processes.process_bigraph import pp, pf
from biosimulator_processes.data_model import *
from biosimulator_processes.biosimulator_builder import BiosimulatorBuilder


model_filepath = '../biosimulator_processes/model_files/BIOMD0000000061_url.xml'

os.path.exists(model_filepath)

CobraProcess registered successfully.
CopasiProcess registered successfully.
TelluriumProcess registered successfully.
DeterministicTimeCourseParameterScan registered successfully.


True

In [1]:
x = {'a': '234', 'b': 20}



In [2]:
from biosimulator_processes.data_model import dynamic_process_config as DPC


config = DPC(config=x, age=29)

CobraProcess registered successfully.
CopasiProcess registered successfully.
TelluriumProcess registered successfully.
DeterministicTimeCourseParameterScan registered successfully.


In [ ]:
human_config = {
    'species': 'human',
    'origin': 'USA'}


rose_config = DPC(
    name='rose',
    config=human_config,
    age=29,
    sibilings=1,
    car='Toyota'
)

In [ ]:
rose_config